In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install python-multipart
!pip install pyngrok

In [2]:
from transformers import pipeline
import soundfile as sf
from fastapi import FastAPI, UploadFile, File, Form
import nest_asyncio
import uvicorn
from pyngrok import ngrok, conf
import datetime

2025-05-09 08:51:11.698193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746780672.005175      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746780672.082551      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
app = FastAPI()

In [4]:
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
conf.get_default().auth_token = ""

In [6]:
@app.get("/")
async def ttt():
  print('get 200')
  return 200

@app.post("/get_data/")
async def get_data(audio: UploadFile = File(...), language: str = Form(...)):
    if language == "english_b": language = "english"
    data, samplerate = sf.read(audio.file)
    sf.write('new_file.wav', data, samplerate)
    text = pipe('new_file.wav', generate_kwargs={"language": language})["text"]
    print(text, language, datetime.datetime.now())
    return {"text": text}

In [ ]:
port = 5000

ngrok.kill()
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel "{public_url}" -> "http://127.0.0.1:{port}"')

nest_asyncio.apply()
uvicorn.run(app, port=port)